In [2]:
import pandas as pd
import numpy as np
import model
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report
import numpy as np
from collections import defaultdict
from model import HART, mobileHART_XS
tf.keras.backend.set_image_data_format('channels_last')

2024-07-18 10:28:03.348915: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-18 10:28:03.822585: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
! pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [4]:
columns = ['Accel X (g)', 'Accel Y (g)', 'Accel Z (g)',
           'Gyro X (°/s)', 'Gyro Y (°/s)', 'Gyro Z (°/s)', 'Activity_Label']
df = pd.read_csv("Final_Resampled_HAR_And_Inertia.csv", header=0, names=columns)

In [5]:
labels = pd.factorize(df['Activity_Label'])[0]

feature_columns = df.columns[:-1]
data = df[feature_columns].values

In [6]:
input_shape = (128, 6)
activityCount = 10

HART = model.HART(input_shape, activityCount)
MobileHART = mobileHART_XS(input_shape, activityCount)

2024-07-18 10:28:19.900434: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-18 10:28:19.900629: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-18 10:28:19.919958: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [12]:
def create_overlapping_windows(data, window_size, step_size, features):
    segments = []
    for start in range(0, len(data) - window_size + 1, step_size):
        segment = data[start:start + window_size, :features]
        segments.append(segment)
    return np.array(segments)

window_size = 128
step_size = 64
num_features = 6

reshaped_data = create_overlapping_windows(data, window_size, step_size, num_features)

def assign_labels_to_windows(labels, window_size, step_size):
    window_labels = []
    for start in range(0, len(labels) - window_size + 1, step_size):
        window_label = np.bincount(labels[start:start + window_size]).argmax()
        window_labels.append(window_label)
    return np.array(window_labels)

window_labels = assign_labels_to_windows(labels, window_size, step_size)

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(reshaped_data, window_labels, test_size=0.2, random_state=21)

In [14]:
HART.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',  
    patience=5,         
    restore_best_weights=True  
)

In [16]:
history = HART.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test),callbacks=[early_stopping])

Epoch 1/200
  7/621 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9837 - loss: 0.0543     

I0000 00:00:1721313609.096818  185318 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_slice_fusion_2', 4 bytes spill stores, 4 bytes spill loads



621/621 ━━━━━━━━━━━━━━━━━━━━ 87s 67ms/step - accuracy: 0.9827 - loss: 0.0577 - val_accuracy: 0.9859 - val_loss: 0.0508
Epoch 2/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9852 - loss: 0.0499 - val_accuracy: 0.9853 - val_loss: 0.0550
Epoch 3/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9873 - loss: 0.0468 - val_accuracy: 0.9896 - val_loss: 0.0394
Epoch 4/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9835 - loss: 0.0580 - val_accuracy: 0.9889 - val_loss: 0.0410
Epoch 5/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9888 - loss: 0.0372 - val_accuracy: 0.9879 - val_loss: 0.0472
Epoch 6/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9889 - loss: 0.0372 - val_accuracy: 0.9872 - val_loss: 0.0439
Epoch 7/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9878 - loss: 0.0379 - val_accuracy: 0.9835 - val_loss: 0.0498
Epoch 8/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9861 - loss: 0.0398 - val_accuracy: 0.98

In [17]:
performance = HART.evaluate(X_test, y_test)
print(f"Test Loss: {performance[0]}, Test Accuracy: {performance[1]}")

I0000 00:00:1721313737.884014  186917 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_2210', 4 bytes spill stores, 4 bytes spill loads



311/311 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9891 - loss: 0.0349
Test Loss: 0.03103194758296013, Test Accuracy: 0.9907351732254028


In [18]:
y_pred_prob = HART.predict(X_test)  
y_pred = np.argmax(y_pred_prob, axis=1)  

report = classification_report(y_test, y_pred, target_names=[f"Class {i}" for i in range(activityCount)])

print(report)

311/311 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step
              precision    recall  f1-score   support

     Class 0       0.99      0.99      0.99       980
     Class 1       1.00      1.00      1.00       945
     Class 2       1.00      0.99      0.99      1012
     Class 3       1.00      0.99      1.00       985
     Class 4       0.98      1.00      0.99       989
     Class 5       0.99      1.00      1.00       987
     Class 6       0.97      0.99      0.98      1022
     Class 7       0.98      0.98      0.98       990
     Class 8       1.00      0.97      0.98      1002
     Class 9       0.99      0.99      0.99      1018

    accuracy                           0.99      9930
   macro avg       0.99      0.99      0.99      9930
weighted avg       0.99      0.99      0.99      9930



# Testing the trained model on HAR data

In [19]:
test_columns = ['Accel X (g)', 'Accel Y (g)', 'Accel Z (g)', 
           'Gyro X (°/s)', 'Gyro Y (°/s)', 'Gyro Z (°/s)', 
           'Activity_Label']

test_df = pd.read_csv("Final_Resampled_HAR_And_Inertia.csv", header=0, names =test_columns)

test_labels = pd.factorize(test_df['Activity_Label'])[0]

test_feature_columns = test_df.columns[:-1]  
test_data = test_df[test_feature_columns].values

test_reshaped_data = create_overlapping_windows(test_data, window_size, step_size, num_features)

test_window_labels = assign_labels_to_windows(test_labels, window_size, step_size)

test_performance = HART.evaluate(test_reshaped_data, test_window_labels)
print(f"Test Loss: {test_performance[0]}, Test Accuracy: {test_performance[1]}")

1544/1552 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9800 - loss: 0.0583

I0000 00:00:1721313780.400366  187011 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_2210', 16 bytes spill stores, 16 bytes spill loads



1552/1552 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9801 - loss: 0.0581
Test Loss: 0.02030979096889496, Test Accuracy: 0.9935946464538574


In [20]:
test_predictions = HART.predict(test_reshaped_data)
test_predictions = np.argmax(test_predictions, axis=1)

print(classification_report(test_window_labels, test_predictions))

1552/1552 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      4964
           1       1.00      1.00      1.00      4965
           2       1.00      0.99      1.00      4965
           3       1.00      1.00      1.00      4964
           4       0.99      1.00      0.99      4965
           5       1.00      1.00      1.00      4965
           6       0.98      1.00      0.99      4964
           7       0.99      0.99      0.99      4965
           8       1.00      0.98      0.99      4964
           9       0.99      0.99      0.99      4965

    accuracy                           0.99     49646
   macro avg       0.99      0.99      0.99     49646
weighted avg       0.99      0.99      0.99     49646



In [21]:
label_counts = defaultdict(int)
total_counts = defaultdict(int)

for i in range(len(test_window_labels)):
    actual_label = test_window_labels[i]
    predicted_label = test_predictions[i]
    label_counts[f"actual_{actual_label}_predicted_{predicted_label}"] += 1
    total_counts[f"actual{actual_label}"] += 1

data = []
for label, count in label_counts.items():
    actual_label = label.split("_")[1]
    total = total_counts[f"actual{actual_label}"]
    ratio = count / total if total > 0 else 0
    data.append([label, count, total, ratio])

results_df = pd.DataFrame(data, columns=["Label", "Count", "Total", "Ratio"])
results_df.sort_values(by="Ratio", ascending=False, inplace=True)
print(results_df)

                   Label  Count  Total     Ratio
17  actual_5_predicted_5   4963   4965  0.999597
37  actual_1_predicted_1   4960   4965  0.998993
14  actual_4_predicted_4   4960   4965  0.998993
19  actual_6_predicted_6   4951   4964  0.997381
11  actual_3_predicted_3   4941   4964  0.995367
9   actual_2_predicted_2   4926   4965  0.992145
34  actual_9_predicted_9   4923   4965  0.991541
3   actual_0_predicted_0   4919   4964  0.990935
25  actual_7_predicted_7   4919   4965  0.990735
28  actual_8_predicted_8   4866   4964  0.980258
29  actual_8_predicted_7     51   4964  0.010274
33  actual_9_predicted_6     40   4965  0.008056
0   actual_0_predicted_4     35   4964  0.007051
26  actual_7_predicted_6     29   4965  0.005841
31  actual_8_predicted_6     25   4964  0.005036
32  actual_8_predicted_9     20   4964  0.004029
13  actual_3_predicted_4     18   4964  0.003626
10  actual_2_predicted_0     17   4965  0.003424
7   actual_2_predicted_4     16   4965  0.003223
24  actual_7_predict

In [22]:
print ('''Tweaked Classification Report for HART trained on HAR dataset''')
results_df.head(20)

Tweaked Classification Report for HART trained on HAR dataset


,Label,Count,Total,Ratio
17,actual_5_predicted_5,4963,4965,0.999597
37,actual_1_predicted_1,4960,4965,0.998993
14,actual_4_predicted_4,4960,4965,0.998993
19,actual_6_predicted_6,4951,4964,0.997381
11,actual_3_predicted_3,4941,4964,0.995367
9,actual_2_predicted_2,4926,4965,0.992145
34,actual_9_predicted_9,4923,4965,0.991541
3,actual_0_predicted_0,4919,4964,0.990935
25,actual_7_predicted_7,4919,4965,0.990735
28,actual_8_predicted_8,4866,4964,0.980258


# HART Trained on HAR and Inertia dataset finetuning on 30% of the Data

In [23]:
columns = ['Accel X (g)', 'Accel Y (g)', 'Accel Z (g)', 
           'Gyro X (°/s)', 'Gyro Y (°/s)', 'Gyro Z (°/s)', 
           'Activity_Label']

df = pd.read_csv("Final_Resampled_HAR_And_Inertia.csv", header=0, names = columns)

from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
df['Activity_Label'] = LE.fit_transform(df['Activity_Label'])

In [24]:
features = df.columns[:-1]  
labels_encoded = df['Activity_Label'].values

data_filtered = df[features].values

window_size = 128
step_size = 64
num_features = 6

reshaped_data_filtered = create_overlapping_windows(data_filtered, window_size, step_size, num_features)

window_labels_filtered = assign_labels_to_windows(labels_encoded, window_size, step_size)

X_fine_tune, X_test_new, y_fine_tune, y_test_new = train_test_split(reshaped_data_filtered, window_labels_filtered, test_size=0.7, random_state=21)

HART.fit(X_fine_tune, y_fine_tune, epochs=200, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/200
186/187 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8398 - loss: 1.1345

I0000 00:00:1721313832.763961  187101 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_21996', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1721313833.368138  187104 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_60', 172 bytes spill stores, 172 bytes spill loads

I0000 00:00:1721313834.891016  187098 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_110', 140 bytes spill stores, 140 bytes spill loads



187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - accuracy: 0.8403 - loss: 1.1302

I0000 00:00:1721313855.717151  187208 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_2210', 72 bytes spill stores, 120 bytes spill loads



187/187 ━━━━━━━━━━━━━━━━━━━━ 39s 210ms/step - accuracy: 0.8408 - loss: 1.1260 - val_accuracy: 0.9832 - val_loss: 0.0607
Epoch 2/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9740 - loss: 0.0926 - val_accuracy: 0.9896 - val_loss: 0.0301
Epoch 3/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9897 - loss: 0.0392 - val_accuracy: 0.9919 - val_loss: 0.0225
Epoch 4/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9924 - loss: 0.0308 - val_accuracy: 0.9876 - val_loss: 0.0357
Epoch 5/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9899 - loss: 0.0354 - val_accuracy: 0.9930 - val_loss: 0.0206
Epoch 6/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9908 - loss: 0.0314 - val_accuracy: 0.9930 - val_loss: 0.0202
Epoch 7/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9918 - loss: 0.0258 - val_accuracy: 0.9923 - val_loss: 0.0214
Epoch 8/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9931 - loss: 0.0232 - val_accuracy: 0.9

In [25]:
performance_new = HART.evaluate(X_test_new, y_test_new)
print(f"Test Loss: {performance_new[0]}, Test Accuracy: {performance_new[1]}")

1087/1087 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9900 - loss: 0.0341
Test Loss: 0.02654147520661354, Test Accuracy: 0.9923459887504578


# HART Trained on HAR and Inertia dataset finetuning on 80% of the Data

In [26]:
columns = ['Accel X (g)', 'Accel Y (g)', 'Accel Z (g)', 
           'Gyro X (°/s)', 'Gyro Y (°/s)', 'Gyro Z (°/s)', 
           'Activity_Label']
df = pd.read_csv("Final_Resampled_HAR_And_Inertia.csv", header=0, names=columns)
labels = pd.factorize(df['Activity_Label'])[0]

feature_columns = df.columns[:-1]  
data = df[feature_columns].values

def create_overlapping_windows(data, window_size, step_size, features):
    segments = []
    for start in range(0, len(data) - window_size + 1, step_size):
        segment = data[start:start + window_size, :features]
        segments.append(segment)
    return np.array(segments)

window_size = 128  
step_size = 64    
num_features = 6 

reshaped_data = create_overlapping_windows(data, window_size, step_size, num_features)


def assign_labels_to_windows(labels, window_size, step_size):
    window_labels = []
    for start in range(0, len(labels) - window_size + 1, step_size):
        window_label = np.bincount(labels[start:start + window_size]).argmax()
        window_labels.append(window_label)
    return np.array(window_labels)

window_labels = assign_labels_to_windows(labels, window_size, step_size)


X_train, X_test, y_train, y_test = train_test_split(reshaped_data, window_labels, test_size=0.2, random_state=42)

HART.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = HART.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/200
  7/621 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.4023 - loss: 6.6273     

I0000 00:00:1721313945.142357  185319 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_slice_fusion_2', 4 bytes spill stores, 4 bytes spill loads



621/621 ━━━━━━━━━━━━━━━━━━━━ 88s 67ms/step - accuracy: 0.8935 - loss: 0.5675 - val_accuracy: 0.9930 - val_loss: 0.0232
Epoch 2/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9887 - loss: 0.0360 - val_accuracy: 0.9934 - val_loss: 0.0210
Epoch 3/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9895 - loss: 0.0359 - val_accuracy: 0.9914 - val_loss: 0.0245
Epoch 4/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9904 - loss: 0.0293 - val_accuracy: 0.9933 - val_loss: 0.0209
Epoch 5/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9913 - loss: 0.0276 - val_accuracy: 0.9878 - val_loss: 0.0363
Epoch 6/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9907 - loss: 0.0309 - val_accuracy: 0.9930 - val_loss: 0.0289
Epoch 7/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9915 - loss: 0.0260 - val_accuracy: 0.9927 - val_loss: 0.0232
Epoch 8/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9912 - loss: 0.0262 - val_accuracy: 0.99

In [27]:
columns = ['Accel X (g)', 'Accel Y (g)', 'Accel Z (g)', 
           'Gyro X (°/s)', 'Gyro Y (°/s)', 'Gyro Z (°/s)', 
           'Activity_Label']
df = pd.read_csv("Final_Resampled_HAR_And_Inertia.csv", header=0, names = columns)

from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
df['Activity_Label'] = LE.fit_transform(df['Activity_Label'])

In [28]:
features = df.columns[:-1]  
labels_encoded = df['Activity_Label'].values

data_filtered = df[features].values

window_size = 128
step_size = 64
num_features = 6

reshaped_data_filtered = create_overlapping_windows(data_filtered, window_size, step_size, num_features)

window_labels_filtered = assign_labels_to_windows(labels_encoded, window_size, step_size)

X_fine_tune, X_test_new, y_fine_tune, y_test_new = train_test_split(reshaped_data_filtered, window_labels_filtered, test_size=0.2, random_state=21)

HART.fit(X_fine_tune, y_fine_tune, epochs=200, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/200
493/497 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9495 - loss: 0.3228

I0000 00:00:1721314059.661984  188114 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_21996', 92 bytes spill stores, 92 bytes spill loads

I0000 00:00:1721314060.719751  188104 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_22245', 24 bytes spill stores, 24 bytes spill loads

I0000 00:00:1721314062.038159  188112 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_22245', 32 bytes spill stores, 32 bytes spill loads

I0000 00:00:1721314062.245652  188104 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_22245', 32 bytes spill stores, 32 bytes spill loads



497/497 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.9498 - loss: 0.3209

I0000 00:00:1721314085.108826  188232 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_8', 284 bytes spill stores, 292 bytes spill loads



497/497 ━━━━━━━━━━━━━━━━━━━━ 45s 91ms/step - accuracy: 0.9498 - loss: 0.3205 - val_accuracy: 0.9926 - val_loss: 0.0251
Epoch 2/200
497/497 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9910 - loss: 0.0309 - val_accuracy: 0.9947 - val_loss: 0.0191
Epoch 3/200
497/497 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9923 - loss: 0.0247 - val_accuracy: 0.9948 - val_loss: 0.0182
Epoch 4/200
497/497 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9921 - loss: 0.0251 - val_accuracy: 0.9941 - val_loss: 0.0174
Epoch 5/200
497/497 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9925 - loss: 0.0250 - val_accuracy: 0.9945 - val_loss: 0.0174
Epoch 6/200
497/497 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9919 - loss: 0.0246 - val_accuracy: 0.9956 - val_loss: 0.0158
Epoch 7/200
497/497 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9904 - loss: 0.0297 - val_accuracy: 0.9961 - val_loss: 0.0130
Epoch 8/200
497/497 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9933 - loss: 0.0209 - val_accuracy: 0.99

In [29]:
performance_new = HART.evaluate(X_test_new, y_test_new)
print(f"Test Loss: {performance_new[0]}, Test Accuracy: {performance_new[1]}")

311/311 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9917 - loss: 0.0233
Test Loss: 0.021205853670835495, Test Accuracy: 0.9928499460220337


# HART on Smote Data 

In [30]:
columns = ['Accel X (g)', 'Accel Y (g)', 'Accel Z (g)', 
           'Gyro X (°/s)', 'Gyro Y (°/s)', 'Gyro Z (°/s)', 
           'Activity_Label']
df1 = pd.read_csv("Final_Resampled_HAR_And_Inertia.csv", header=0, names=columns)

labels = pd.factorize(df1['Activity_Label'])[0]

feature_columns = df1.columns[:-1] 
data = df1[feature_columns].values

In [31]:
window_size = 128 
step_size = 64    
num_features = 6 

reshaped_data = create_overlapping_windows(data, window_size, step_size, num_features)

window_labels = assign_labels_to_windows(labels, window_size, step_size)

input_shape = (128, 6)
activityCount = len(np.unique(labels)) 

In [32]:
HART = model.HART(input_shape, activityCount)

X_train, X_test, y_train, y_test = train_test_split(reshaped_data, window_labels, test_size=0.2, random_state=42)

HART.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

In [33]:
history = HART.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/200
 13/621 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.2028 - loss: 1.9399   

I0000 00:00:1721314249.405152  185317 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_slice_fusion_2', 4 bytes spill stores, 4 bytes spill loads



621/621 ━━━━━━━━━━━━━━━━━━━━ 87s 66ms/step - accuracy: 0.4344 - loss: 1.2792 - val_accuracy: 0.7055 - val_loss: 0.7057
Epoch 2/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.7216 - loss: 0.6855 - val_accuracy: 0.7518 - val_loss: 0.5954
Epoch 3/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8006 - loss: 0.5196 - val_accuracy: 0.9575 - val_loss: 0.2429
Epoch 4/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9187 - loss: 0.2723 - val_accuracy: 0.9709 - val_loss: 0.1471
Epoch 5/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9365 - loss: 0.2029 - val_accuracy: 0.9787 - val_loss: 0.1196
Epoch 6/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9439 - loss: 0.1750 - val_accuracy: 0.9783 - val_loss: 0.0984
Epoch 7/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9498 - loss: 0.1507 - val_accuracy: 0.9777 - val_loss: 0.0936
Epoch 8/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9602 - loss: 0.1251 - val_accuracy: 0.98

In [34]:
performance = HART.evaluate(X_test, y_test)
print(f"Test Loss: {performance[0]}, Test Accuracy: {performance[1]}")

311/311 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9917 - loss: 0.0248
Test Loss: 0.02601544000208378, Test Accuracy: 0.991540789604187


In [35]:
test_predictions = HART.predict(test_reshaped_data)
test_predictions = np.argmax(test_predictions, axis=1)

print(classification_report(test_window_labels, test_predictions))

1552/1552 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4964
           1       1.00      1.00      1.00      4965
           2       1.00      0.99      0.99      4965
           3       1.00      1.00      1.00      4964
           4       0.99      1.00      0.99      4965
           5       1.00      1.00      1.00      4965
           6       1.00      0.99      0.99      4964
           7       0.99      1.00      0.99      4965
           8       1.00      0.99      0.99      4964
           9       0.99      1.00      0.99      4965

    accuracy                           0.99     49646
   macro avg       0.99      0.99      0.99     49646
weighted avg       0.99      0.99      0.99     49646

